### Columns present in ML model - obtained from Project 2 - 3. Model training (OHE for bases + No relative positions)
`['Mean SD', 'Mean_Mean', 'Mean Dwelling Time', 'AAACA', 'AAACC', 'AAACT', 'AACAA', 'AACAC',
  'AACAG', 'AACAT', 'AACCA', 'AACCC', 'AACCG', 'AACCT', 'AACTA', 'AACTC', 'AACTG', 'AACTT', 'AAGAC', 'AGAAC', 'AGACA', 'AGACC',   'AGACT', 'AGGAC', 'ATAAC', 'ATGAC', 'CAAAC', 'CAGAC', 'CGAAC', 'CGGAC', 'CTAAC', 'CTGAC', 'GAAAC', 'GAACA', 'GAACC', 'GAACT',   'GACAA', 'GACAC', 'GACAG', 'GACAT', 'GACCA', 'GACCC', 'GACCG', 'GACCT', 'GACTA', 'GACTC', 'GACTG', 'GACTT', 'GAGAC', 'GGAAC',   'GGACA', 'GGACC', 'GGACT', 'GGGAC', 'GTAAC', 'GTGAC', 'TAAAC', 'TAACA', 'TAACC', 'TAACT', 'TAGAC', 'TGAAC', 'TGACA', 'TGACC',
  'TGACT', 'TGGAC', 'TTAAC', 'TTGAC', 'Top Bases Indicator', 'Bottom Bases Indicator', 'Count_A', 'Count_T', 'Count_C']`

In [1]:
import pickle
import pandas as pd
df = pd.read_csv('./Data/dataset2.csv')
df

,Transcript Name,Position,Bases,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time
0,AT1G01050.1,154,GAAAC,2.557500,109.888889,0.006574,2.090,110.00,0.005310
1,AT1G01050.1,155,AAACT,2.278611,108.716667,0.007237,2.130,110.00,0.006360
2,AT1G01050.1,156,AACTA,1.804167,95.641667,0.006340,1.720,95.50,0.005585
3,AT1G01050.1,164,ATAAC,2.715526,84.363158,0.006866,2.465,84.35,0.005705
4,AT1G01050.1,165,TAACC,3.054474,93.942105,0.006421,3.030,93.80,0.006130
...,...,...,...,...,...,...,...,...,...
296485,AT5G67590.1,663,AAACC,3.918317,99.866337,0.005711,3.890,99.40,0.004960
296486,AT5G67590.1,664,AACCT,2.483366,82.502970,0.006016,2.380,82.50,0.005030
296487,AT5G67600.1,153,AGAAC,6.446333,132.900000,0.008041,6.425,134.00,0.007360
296488,AT5G67600.1,154,GAACT,3.094667,102.260000,0.006118,2.330,102.00,0.005605


In [2]:
indicator = pd.DataFrame([0,1,0] * int(len(df) / 3), columns = ['Indicator'])
df = pd.concat([df, indicator], axis=1)
df = df[df['Indicator'] == 1]
df = df.drop(['Indicator'], axis=1).reset_index(drop=True)
transcript_name_and_position = df[['Transcript Name', 'Position']]
df = df.drop(['Transcript Name','Position'], axis=1)
df

,Bases,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time
0,AAACT,2.278611,108.716667,0.007237,2.130,110.0,0.006360
1,TAACC,3.054474,93.942105,0.006421,3.030,93.8,0.006130
2,AGACA,7.898000,124.514286,0.009519,7.140,125.0,0.007970
3,AGACA,5.626829,128.314286,0.007980,5.590,129.0,0.006310
4,AGACA,5.744500,128.400000,0.009057,5.475,129.0,0.007470
...,...,...,...,...,...,...,...
98825,AAACT,2.874530,108.358120,0.008172,2.740,109.0,0.005980
98826,TGACC,7.781062,119.141593,0.005314,7.690,120.0,0.004320
98827,TGACT,7.363898,121.949153,0.006042,7.350,122.0,0.005415
98828,AAACC,3.918317,99.866337,0.005711,3.890,99.4,0.004960


### Pre-processing data to make sure that data has columns that are accepted by ML model

In [3]:
# From the EDA, it appears that bases ['GGACT', 'GAACT', 'GGACA', 'GGACC', 'AGACT', 'TGACT'] appear the highest amount of 
# times in m6a positive sites - as such, I will use a greater weight when those bases are present
def indicate_top_bases(bases):
    if bases in ['GGACT', 'GAACT']:
        return 2
    elif bases in ['GGACA', 'GGACC', 'AGACT', 'TGACT']:
        return 1
    return 0

# From the EDA, it appears that bases ['GGACT', 'GAACT', 'GGACA', 'GGACC', 'AGACT', 'TGACT'] appear the highest amount of 
# times in m6a negative sites - as such, I will use a smaller weight when those bases are present
def indicate_bottom_bases(bases):
    if bases in ['AAACT', 'GAACA', 'TGACA', 'AGACA', 'TGACC', 'GGACA']:
        return -1
    return 0

# Obtain the counts of the individual bases and use them as features
def count_bases(bases):
    a,t,c=0,0,0
    for i in bases:
        if i == 'A':
            a+=1
        elif i == 'T':
            t+=1
        elif i == 'C':
            c+=1
    return a,t,c

In [4]:
df['Top Bases Indicator'] = df['Bases'].apply(indicate_top_bases)
df['Bottom Bases Indicator'] = df['Bases'].apply(indicate_bottom_bases)
df['Combined Bases Indicator'] = df['Top Bases Indicator'] + df['Bottom Bases Indicator']
df['Count_A'], df['Count_T'], df['Count_C'] = zip(*df['Bases'].apply(count_bases))
df

,Bases,Mean SD,Mean_Mean,Mean Dwelling Time,Median SD,Median_Mean,Median Dwelling Time,Top Bases Indicator,Bottom Bases Indicator,Combined Bases Indicator,Count_A,Count_T,Count_C
0,AAACT,2.278611,108.716667,0.007237,2.130,110.0,0.006360,0,-1,-1,3,1,1
1,TAACC,3.054474,93.942105,0.006421,3.030,93.8,0.006130,0,0,0,2,1,2
2,AGACA,7.898000,124.514286,0.009519,7.140,125.0,0.007970,0,-1,-1,3,0,1
3,AGACA,5.626829,128.314286,0.007980,5.590,129.0,0.006310,0,-1,-1,3,0,1
4,AGACA,5.744500,128.400000,0.009057,5.475,129.0,0.007470,0,-1,-1,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98825,AAACT,2.874530,108.358120,0.008172,2.740,109.0,0.005980,0,-1,-1,3,1,1
98826,TGACC,7.781062,119.141593,0.005314,7.690,120.0,0.004320,0,-1,-1,1,1,2
98827,TGACT,7.363898,121.949153,0.006042,7.350,122.0,0.005415,1,0,1,1,2,1
98828,AAACC,3.918317,99.866337,0.005711,3.890,99.4,0.004960,0,0,0,3,0,2


### Remove columns based on Pearson Correlation 
Refer to `Project 2 - 3. Model training (OHE for bases + No relative positions)`

In [5]:
df = df.drop(['Median Dwelling Time', 'Median SD', 'Median_Mean', 'Combined Bases Indicator'], axis=1)
df_dummies = pd.get_dummies(df['Bases'], drop_first=True)
df = pd.concat([df, df_dummies],axis=1).drop(['Bases'],axis=1)
df

,Mean SD,Mean_Mean,Mean Dwelling Time,Top Bases Indicator,Bottom Bases Indicator,Count_A,Count_T,Count_C,AAACC,AAACT,...,GAACT,GGACA,GGACC,GGACT,TAACA,TAACC,TAACT,TGACA,TGACC,TGACT
0,2.278611,108.716667,0.007237,0,-1,3,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,3.054474,93.942105,0.006421,0,0,2,1,2,0,0,...,0,0,0,0,0,1,0,0,0,0
2,7.898000,124.514286,0.009519,0,-1,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5.626829,128.314286,0.007980,0,-1,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5.744500,128.400000,0.009057,0,-1,3,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98825,2.874530,108.358120,0.008172,0,-1,3,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
98826,7.781062,119.141593,0.005314,0,-1,1,1,2,0,0,...,0,0,0,0,0,0,0,0,1,0
98827,7.363898,121.949153,0.006042,1,0,1,2,1,0,0,...,0,0,0,0,0,0,0,0,0,1
98828,3.918317,99.866337,0.005711,0,0,3,0,2,1,0,...,0,0,0,0,0,0,0,0,0,0


### Import ML Model to predict on test dataset

In [6]:
pickled_model = pickle.load(open('gradientboostingclassifier.sav', 'rb'))
predicted_scores = pickled_model.predict_proba(df)
predicted_scores

C:\Users\jingh\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


array([[0.09189962, 0.90810038],
       [0.01887936, 0.98112064],
       [0.42952478, 0.57047522],
       ...,
       [0.61245542, 0.38754458],
       [0.12230243, 0.87769757],
       [0.56798682, 0.43201318]])

In [7]:
predicted_full_df = pd.concat([transcript_name_and_position, pd.DataFrame(predicted_scores[:,1], columns = ['score'])], axis=1)
predicted_full_df['transcript_id'], predicted_full_df['transcript_position'] = predicted_full_df['Transcript Name'], predicted_full_df['Position']
predicted_full_df = predicted_full_df.drop(['Transcript Name', 'Position'], axis=1)
predicted_full_df = predicted_full_df[['transcript_id', 'transcript_position', 'score']]
predicted_full_df

,transcript_id,transcript_position,score
0,AT1G01050.1,155,0.908100
1,AT1G01050.1,165,0.981121
2,AT1G01050.1,347,0.570475
3,AT1G01050.1,435,0.162919
4,AT1G01050.2,463,0.199604
...,...,...,...
98825,AT5G67590.1,367,0.920086
98826,AT5G67590.1,444,0.468469
98827,AT5G67590.1,465,0.387545
98828,AT5G67590.1,663,0.877698


In [8]:
predicted_full_df.to_csv('./Data/dataset2_prediction_2.csv', index=False)